In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1940,2024,1)

ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

areas_array_era5_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_era5_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_era5_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]

areas_list_era5_polar=[]
areas_list_era5_fixed=[]
areas_list_era5_fixed_poleward=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    #################
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    
    if i==0:#land mask option
        ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
        t2m=ds['t2m'][0,:,:]
        land_mask=t2m/t2m
        land_mask=land_mask.fillna(0)
        
        lon_east=[x for x in land_mask.longitude.values if x>=180.]
        lon_west=[x for x in land_mask.longitude.values if x<180.]
        l_east=land_mask.sel(longitude=lon_east)
        l_west=land_mask.sel(longitude=lon_west)
        l_east['longitude']=[x-360. for x in lon_east]
        land_mask=xr.concat([l_east,l_west],dim='longitude')
        
        dk=xr.Dataset()
        dk['land']=(('lat','lon'),land_mask.values)
        dk.coords['lat']=land_mask.latitude.values
        dk.coords['lon']=land_mask.longitude.values
        
        land_mask=dk['land']
        land_mask_interp=land_mask.interp_like(labeled[0,:,:])
        
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        
        if 1==0:
            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,label_sel,vmin=0,vmax=label_sel.max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()

            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,labeled[j,:,:],vmin=0,vmax=labeled[j,:,:].max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()
            sys.exit()
        
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_era5_fixed.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_era5_fixed[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    ################

    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_era5_polar.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_era5_polar[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    #################
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_era5_fixed_poleward.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_era5_fixed_poleward[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
print('DONE')

In [ ]:
yrs=np.arange(1958,2024,1)

ds=xr.open_dataset(dir3+'jra55_gridareas.nc')
areas=ds['cell_area']/1e6

areas_array_jra55_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_jra55_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_jra55_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]

areas_list_jra55_polar=[]
areas_list_jra55_fixed=[]
areas_list_jra55_fixed_poleward=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    #################
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    areas['lat']=labeled.lat.values
    areas['lon']=labeled.lon.values
    
    if i==0:#land mask option
        ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
        t2m=ds['t2m'][0,:,:]
        land_mask=t2m/t2m
        land_mask=land_mask.fillna(0)
        
        lon_east=[x for x in land_mask.longitude.values if x>=180.]
        lon_west=[x for x in land_mask.longitude.values if x<180.]
        l_east=land_mask.sel(longitude=lon_east)
        l_west=land_mask.sel(longitude=lon_west)
        l_east['longitude']=[x-360. for x in lon_east]
        land_mask=xr.concat([l_east,l_west],dim='longitude')
        
        dk=xr.Dataset()
        dk['land']=(('lat','lon'),land_mask.values)
        dk.coords['lat']=land_mask.latitude.values
        dk.coords['lon']=land_mask.longitude.values
        
        land_mask=dk['land']
        land_mask_interp=land_mask.interp_like(labeled[0,:,:])
        
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        
        if 1==0:
            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,label_sel,vmin=0,vmax=label_sel.max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()

            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,labeled[j,:,:],vmin=0,vmax=labeled[j,:,:].max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()
            sys.exit()
        
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_jra55_fixed.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_jra55_fixed[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    ################

    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    areas['lat']=labeled.lat.values
    areas['lon']=labeled.lon.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_jra55_polar.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_jra55_polar[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    #################
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    areas['lat']=labeled.lat.values
    areas['lon']=labeled.lon.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_jra55_fixed_poleward.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_jra55_fixed_poleward[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
print('DONE')

In [ ]:
yrs=np.arange(1980,2024,1)

ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

areas_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]

areas_list_merra2_polar=[]
areas_list_merra2_fixed=[]
areas_list_merra2_fixed_poleward=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    #################
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    
    if i==0:#land mask option
        ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
        t2m=ds['t2m'][0,:,:]
        land_mask=t2m/t2m
        land_mask=land_mask.fillna(0)
        
        lon_east=[x for x in land_mask.longitude.values if x>=180.]
        lon_west=[x for x in land_mask.longitude.values if x<180.]
        l_east=land_mask.sel(longitude=lon_east)
        l_west=land_mask.sel(longitude=lon_west)
        l_east['longitude']=[x-360. for x in lon_east]
        land_mask=xr.concat([l_east,l_west],dim='longitude')
        
        dk=xr.Dataset()
        dk['land']=(('lat','lon'),land_mask.values)
        dk.coords['lat']=land_mask.latitude.values
        dk.coords['lon']=land_mask.longitude.values
        
        land_mask=dk['land']
        land_mask_interp=land_mask.interp_like(labeled[0,:,:])
        
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        
        if 1==0:
            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,label_sel,vmin=0,vmax=label_sel.max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()

            fig=plt.figure(figsize=(20,10))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            ax.pcolormesh(label_sel.lon,label_sel.lat,labeled[j,:,:],vmin=0,vmax=labeled[j,:,:].max().values,cmap=plt.cm.viridis)
            ax.coastlines(resolution='10m')
            plt.show()
            sys.exit()
        
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_merra2_fixed.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_merra2_fixed[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    ################

    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_merra2_polar.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_merra2_polar[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
        
    #################
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp['lon']=labeled.lon.values
    land_mask_interp['lat']=labeled.lat.values
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    area_sum_year=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:].where(land_mask_interp>=0.5)
        area_day=areas.where(label_sel>0).sum().values.tolist()
        area_sum_year=area_sum_year+area_day
        areas_array_year[month-1].append(area_day)
            
    mean_area=area_sum_year/float(labeled.time.size)#km2 per day
    areas_list_merra2_fixed_poleward.append(mean_area)

    for j in range(len(areas_array_year)):
        areas_array_merra2_fixed_poleward[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month     
print('DONE')

In [ ]:
dk=xr.Dataset()

dk['areas_array_era5_fixed_poleward']=(('months','years'),np.asarray(areas_array_era5_fixed_poleward))
dk['areas_array_era5_fixed']=(('months','years'),np.asarray(areas_array_era5_fixed))
dk['areas_array_era5_variable']=(('months','years'),np.asarray(areas_array_era5_polar))

dk['areas_list_era5_variable']=(('years'),areas_list_era5_polar)
dk['areas_list_era5_fixed']=(('years'),areas_list_era5_fixed)
dk['areas_list_era5_fixed_poleward']=(('years'),areas_list_era5_fixed_poleward)


dk['areas_array_merra2_fixed_poleward']=(('months','years2'),np.asarray(areas_array_merra2_fixed_poleward))
dk['areas_array_merra2_fixed']=(('months','years2'),np.asarray(areas_array_merra2_fixed))
dk['areas_array_merra2_variable']=(('months','years2'),np.asarray(areas_array_merra2_polar))

dk['areas_list_merra2_variable']=(('years2'),areas_list_merra2_polar)
dk['areas_list_merra2_fixed']=(('years2'),areas_list_merra2_fixed)
dk['areas_list_merra2_fixed_poleward']=(('years2'),areas_list_merra2_fixed_poleward)


dk['areas_array_jra55_fixed_poleward']=(('months','years1'),np.asarray(areas_array_jra55_fixed_poleward))
dk['areas_array_jra55_fixed']=(('months','years1'),np.asarray(areas_array_jra55_fixed))
dk['areas_array_jra55_variable']=(('months','years1'),np.asarray(areas_array_jra55_polar))

dk['areas_list_jra55_variable']=(('years1'),areas_list_jra55_polar)
dk['areas_list_jra55_fixed']=(('years1'),areas_list_jra55_fixed)
dk['areas_list_jra55_fixed_poleward']=(('years1'),areas_list_jra55_fixed_poleward)

dk.coords['years']=np.arange(1940,2024,1)
dk.coords['years1']=np.arange(1958,2024,1)
dk.coords['years2']=np.arange(1980,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_area_land.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_area_land.nc',mode='w',format='NETCDF4')